In [1]:
import pandas as pd

# 엑셀 파일 불러오기 (sheet_name으로 특정 시트 지정 가능)
df_price_raw = pd.read_excel("etf_raw_daily.xlsx", sheet_name='asset_price')

In [2]:
df_price = pd.DataFrame(df_price_raw.iloc[16:,])
df_price = df_price.rename(columns=df_price_raw.iloc[12,1:49])

In [3]:
df_price.set_index('Unnamed: 0', inplace=True)

In [4]:
weekly_day_price = df_price[df_price.index.weekday ==4]

In [5]:
asset_return_weekly = weekly_day_price.pct_change()*100

C:\Users\USER\AppData\Local\Temp\ipykernel_17060\3094994716.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  asset_return_weekly = weekly_day_price.pct_change()*100


In [6]:
asset_return_weekly.to_csv('./data/asset_return_weekly.csv')

# col 1_1 자산별 가격 추이

In [8]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go


df = pd.read_csv("./data/asset_return_weekly.csv", index_col=0, parse_dates=True)

df_col1_1 = df

asset_return_1w = df_col1_1.iloc[-52:,:3]/100

index_df = (1 + asset_return_1w).cumprod() * 100

# **100을 기준으로 지수 계산**
index_df = (1 + asset_return_1w).cumprod() * 100

# **첫 번째 행 추가 (기준값 100)**
start_date = asset_return_1w.index[0] - pd.DateOffset(weeks=1)  # 첫 번째 주 이전 날짜 생성
initial_row = pd.DataFrame([[100, 100, 100]], columns=index_df.columns, index=[start_date])  # 100으로 된 행 추가

# **초기 값(100)과 기존 데이터 결합**
asset_index_df = pd.concat([initial_row, index_df])


In [9]:
asset_index_df.to_csv('./data/asset_index_df.csv')

In [10]:
df_test = pd.read_csv("./data/asset_index_df.csv", index_col=0, parse_dates=True)


In [11]:
import streamlit as st
import pandas as pd
import altair as alt

# 데이터 로드 함수
@st.cache_data
def load_data():
    df = pd.read_csv("./data/asset_return_weekly.csv", index_col=0, parse_dates=True)
    return df

# 데이터 불러오기
df = load_data()
df = round(df, 2)  # 소수점 두 자리 반올림

# 최근 12주 데이터 필터링 (모든 자산 포함)
filtered_df = df.tail(12).T
filtered_df = filtered_df.loc[["Stock", "Bond", "Commodity"], :]  # 순서 유지

# 데이터 변환: Altair를 위한 DataFrame 구조 변경
filtered_df = filtered_df.reset_index().melt(id_vars="index", var_name="Date", value_name="Return")
filtered_df.rename(columns={"index": "Asset"}, inplace=True)

# ✅ "Asset" 컬럼을 Categorical 타입으로 변환하여 순서 유지
asset_order = ["Stock", "Bond", "Commodity"]
filtered_df["Asset"] = pd.Categorical(filtered_df["Asset"], categories=asset_order, ordered=True)

# 날짜 포맷 변경 (YYYY-MM-DD → Jan 15 형식)
filtered_df["Date"] = pd.to_datetime(filtered_df["Date"]).dt.strftime("%b %d")

# ✅ 폰트 크기 및 스타일 설정 (모든 텍스트에 적용)
font_size = 15  # 공통 폰트 크기 설정
font_family = "Times New Roman"  # 원하는 폰트 (예: 'Times New Roman', 'Courier New')

# 히트맵 생성 (색상 조정 + 값 입력 추가)
heatmap = alt.Chart(filtered_df).mark_rect(stroke="black", strokeWidth=0.25).encode(
    x=alt.X("Date:O", title="Date", axis=alt.Axis(labelFontSize=font_size, titleFontSize=font_size, labelFont=font_family, titleFont=font_family)),
    y=alt.Y("Asset:O", title="", sort=asset_order, axis=alt.Axis(labelFontSize=font_size, titleFontSize=font_size, labelFont=font_family, titleFont=font_family)),
    color=alt.Color(
        "Return:Q",
        scale=alt.Scale(domain=[-5, 0, 5], range=["#ffffff", "#c6dbef", "#084594"]),  # 색상 조정
        title="Return"
    ),
    tooltip=["Date", "Asset", "Return"]
).properties(
    width=900,
    height=400
)

# 데이터 라벨 추가 (각 셀에 숫자 표시, 폰트 크기 및 스타일 적용)
text = alt.Chart(filtered_df).mark_text(baseline="middle", fontSize=font_size, font=font_family).encode(
    x="Date:O",
    y=alt.Y("Asset:O", sort=asset_order),
    text=alt.Text("Return:Q", format=".2f"),  # 소수점 2자리 표시
    color=alt.condition(
        alt.datum.Return > 0, alt.value("#4d4d4d"), alt.value("#4d4d4d")  # 밝은 배경이므로 검정색 글씨 유지
    )
)

# ✅ LayerChart에서 폰트 크기 및 스타일 적용
final_chart = (heatmap + text).configure_axis(
    labelFontSize=font_size,
    titleFontSize=font_size,
    labelFont=font_family,  # 축 라벨 폰트 적용
    titleFont=font_family   # 축 제목 폰트 적용
).configure_title(
    fontSize=font_size + 2,
    font=font_family  # 타이틀 폰트 적용
).configure_legend(
    labelFontSize=font_size,
    titleFontSize=font_size,
    labelFont=font_family,  # 범례 폰트 적용
    titleFont=font_family
).configure_text(
    font=font_family  # 히트맵 안 숫자 폰트 적용
)

# 히트맵 출력 (데이터 라벨 포함)
st.altair_chart(final_chart, use_container_width=True)


2025-02-09 17:59:27.229 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-02-09 17:59:27.230 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-02-09 17:59:27.231 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 17:59:27.351 
  command:

    streamlit run C:\Users\USER\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-09 17:59:27.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 17:59:27.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 17:59:27.360 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-09 17:59:27.361 Thread 'MainThread': missing

DeltaGenerator()

In [12]:
filtered_df.to_csv('./data/filtered_df.csv')

In [13]:
filtered_df

,Asset,Date,Return
0,Stock,Nov 22,1.37
1,Bond,Nov 22,0.19
2,Commodity,Nov 22,3.80
3,Stock,Nov 29,0.98
4,Bond,Nov 29,1.39
5,Commodity,Nov 29,-2.10
6,Stock,Dec 06,1.30
7,Bond,Dec 06,0.45
8,Commodity,Dec 06,-0.79
9,Stock,Dec 13,-0.87
